In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import os
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, random_split
import csv

1. pred.detach().cpu 的用法
2. tensorboard的用法
3. test为什么要变成model.eval()模式？

In [21]:
def select_features(train_data, valid_data, test_data, select_all = True):
    y_train, y_valid  = train_data[:, -1], valid_data[:, -1]

    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data

    if select_all:
        feature_idx = list(range(raw_x_train.shape[1]))
    else:
        feature_idx = [0, 1, 2, 3, 4]

    return raw_x_train[:, feature_idx], raw_x_valid[:, feature_idx], raw_x_test[:, feature_idx], y_train, y_valid

def predict(model, test_loader, device):
    model.eval()
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()

    return preds

def train_valid_split(data_set, valid_ratio):
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size])
    return np.array(train_set), np.array(valid_set)

In [5]:
# 读数据的类
class myData(Dataset):
    def __init__(self, x, y=None):
        # 开预测模式
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        if self.y is None:
            return self.x[index]
        return self.x[index], self.y[index]

In [17]:
import torch
import torch.nn as nn
# network structure
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
    
    def forward(self, x):
        x = self.layers(x)
        return x.squeeze(1)

In [26]:
# 训练
def train(train_loader, valid_loader, config, device):
    criterion = nn.MSELoss()
    # 动态学习率
    optimizer = torch.optim.SGD(model.parameters(), lr = config['learning_rate'], momentum=0.9)

    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')
    
    n_epochs, best_loss, step, early_step_count = config['n_epochs'], float('inf'), 0, 0
    
    # 训练 交叉验证 验证
    # train valid test
    for epoch in range(n_epochs):
        model.train()
        loss_record = []
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
        
        mean_train_loss = sum(loss_record) / len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()
        loss_record = []

        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)
            loss_record.append(loss.item())
        mean_valid_loss = sum(loss_record) / len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}] : Train loss : {mean_train_loss:.4f}, Valid loss : {mean_valid_loss: .4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path'])
            print('Saving model with loss {:3f}...'.format(best_loss))
            early_step_count = 0
        else:
            early_step_count += 1
        
        if early_step_count >= config['early_stop']:
            print('\nM<odel is not improving, so we halt the training session.')
            return 

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 超参数
config = {
    'seed'         : 20020225,
    'select_all'   : True,
    'valid_ratio'  : 0.3,
    'n_epochs'     : 3000,
    'batch_size'   : 256,
    'learning_rate': 1e-5,
    'early_stop'   : 400,
    'save_path'    : './models/model.ckpt'
}

pin_memory 锁页内存，能加快训练速度

In [28]:
same_seed = config['seed']

train_data = pd.read_csv('data/covid.train.csv').values
test_data = pd.read_csv('data/covid.test.csv').values

train_data, valid_data = train_valid_split(train_data, config['valid_ratio'])

x_train, x_valid, x_test, y_train, y_valid = select_features(
    train_data, valid_data, test_data, select_all=config['select_all']
)

train_dataset = myData(x_train, y_train)
valid_dataset = myData(x_valid, y_valid)
test_dataset  = myData(x_test)

train_loader = DataLoader(train_dataset, batch_size = config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size = config['batch_size'], shuffle=True, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size = config['batch_size'], shuffle=True, pin_memory=True)

In [29]:
model = Model(input_dim=x_train.shape[1]).to(device)
train(train_loader, valid_loader, config, device)

Epoch [1/3000]: 100%|██████████| 8/8 [00:00<00:00, 326.14it/s, loss=43.6]


Epoch [1/3000] : Train loss : 46.6931, Valid loss :  45.6297
Saving model with loss 45.629730...


Epoch [2/3000]: 100%|██████████| 8/8 [00:00<00:00, 350.20it/s, loss=39.6]


Epoch [2/3000] : Train loss : 40.7095, Valid loss :  34.3666
Saving model with loss 34.366585...


Epoch [3/3000]: 100%|██████████| 8/8 [00:00<00:00, 322.51it/s, loss=35.7]


...

Epoch [2998/3000] : Train loss : 1.8011, Valid loss :  1.6552


Epoch [2999/3000]: 100%|██████████| 8/8 [00:00<00:00, 422.30it/s, loss=2]


Epoch [2999/3000] : Train loss : 2.1756, Valid loss :  2.0715


Epoch [3000/3000]: 100%|██████████| 8/8 [00:00<00:00, 369.00it/s, loss=1.8]


Epoch [3000/3000] : Train loss : 1.7604, Valid loss :  1.4409


# Plot learning curves with tensorboard

In [10]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 58267), started 0:14:42 ago. (Use '!kill 58267' to kill it.)

# Testing

model.load_state_dict(torch.load(root)) 加载训练好的模型

In [14]:
# def save_pred(preds, file):
#     with open(file, 'w') as f:
#         writer = csv.writer(f)
#         writer.writerow(['id'], 'tested_positive')
#         for i, p in enumerate(preds):
#             writer.writerow([i, p])
def save_pred(preds, file):
    fp = pd.DataFrame(
        preds,
        columns=['tested_positive']
    )
    fp.to_csv(file)

model = Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(model, test_loader, device)
save_pred(preds, 'pred.csv')

  0%|          | 0/4 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x116 and 117x16)